In [1]:
import numpy as np
from clawpack import pyclaw
from clawpack import riemann

def advection_1D(ql, qr, auxl, auxr, problem_data):
    num_eqn = 1  # Number of equations
    num_waves = 1  # Number of waves in the Riemann solution

    # Calculate number of interfaces, which should be the same as the number of cells (periodic boundary)
    num_interfaces = ql.shape[-1]

    # Initialize arrays for waves, speeds, and fluctuations
    wave = np.zeros((num_eqn, num_waves, num_interfaces))
    s = np.zeros((num_waves, num_interfaces))
    amdq = np.zeros((num_eqn, num_interfaces))
    apdq = np.zeros((num_eqn, num_interfaces))

    # Calculate speeds at cell interfaces, correctly handling periodic boundaries
    speed = 1.5 + np.sin(2 * np.pi * problem_data['grid'].x.centers)
    # Append the speed at the first cell to the end to handle periodic boundary
    speed = np.append(speed, speed[0])
    speed_interface = 0.5 * (speed[:-1] + speed[1:])

    # Compute the fluctuations based on the upwind scheme
    for i in range(num_interfaces):
        if speed_interface[i] > 0:
            amdq[:, i] = -speed_interface[i] * ql[:, i]
            apdq[:, i] = speed_interface[i] * qr[:, i]
        else:
            amdq[:, i] = -speed_interface[i] * qr[:, i]
            apdq[:, i] = speed_interface[i] * ql[:, i]

    return wave, s, amdq, apdq

def setup():
    # Define the computational domain
    x_lower = 0.0
    x_upper = 1.0
    num_cells = 100
    x = pyclaw.Dimension(x_lower, x_upper, num_cells, name='x')
    domain = pyclaw.Domain([x])

    # Create a solver object
    solver = pyclaw.ClawSolver1D(riemann_solver=advection_1D)
    solver.kernel_language = 'Python'
    solver.num_eqn = 1
    solver.num_waves = 1

    # Initialize the state
    state = pyclaw.State(domain, solver.num_eqn)
    state.problem_data['grid'] = domain.grid  # Pass grid info to problem data for access in Riemann solver
    xc = domain.grid.x.centers
    state.q[0, :] = np.exp(-100 * (xc - 0.5)**2)  # Initial condition

    # Boundary conditions
    solver.bc_lower[0] = pyclaw.BC.periodic
    solver.bc_upper[0] = pyclaw.BC.periodic

    # Controller setup
    claw = pyclaw.Controller()
    claw.tfinal = 1.0
    claw.solution = pyclaw.Solution(state, domain)
    claw.solver = solver
    claw.keep_copy = True  # Keep a copy of the solution
    claw.output_style = 1  # Output at every timestep

    return claw

if __name__ == '__main__':
    claw = setup()
    claw.run()

    import matplotlib.pyplot as plt

    for frame in claw.frames:
        plt.plot(frame.state.grid.x.centers, frame.state.q[0, :])
    plt.title("Solution of the Advection Equation")
    plt.xlabel("x")
    plt.ylabel("u")
    plt.show()


********************************************************************
 Did you run "pip install" in your clawpack directory?
********************************************************************
2024-05-10 14:07:05,264 INFO CLAW: Solution 0 computed for time t=0.000000


Traceback (most recent call last):
  File "/Users/hansonmo/opt/anaconda3/lib/python3.9/site-packages/clawpack/riemann/__init__.py", line 56, in <module>
    from . import acoustics_1D
ImportError: dlopen(/Users/hansonmo/opt/anaconda3/lib/python3.9/site-packages/clawpack/riemann/acoustics_1D.cpython-39-darwin.so, 2): no suitable image found.  Did find:
	/Users/hansonmo/opt/anaconda3/lib/python3.9/site-packages/clawpack/riemann/acoustics_1D.cpython-39-darwin.so: mach-o, but wrong architecture
	/Users/hansonmo/opt/anaconda3/lib/python3.9/site-packages/clawpack/riemann/acoustics_1D.cpython-39-darwin.so: mach-o, but wrong architecture


IndexError: index 100 is out of bounds for axis 0 with size 100